라이브러리 로드

In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

데이터셋로드

In [2]:
df = pd.read_csv("data/diabetes_feature.csv")
df.shape

(768, 16)

In [3]:
df.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome,Pregnancies_high,Age_low,Age_middle,Age_high,Insulin_nan,Insulin_log,low_glu_insulin
0,6,0.848324,72,35,0,33.6,0.468492,50,1,False,False,True,False,169.5,5.138735,False
1,1,-1.123396,66,29,0,26.6,-0.365061,31,0,False,False,True,False,102.5,4.639572,True
2,8,1.943724,64,0,0,23.3,0.604397,32,1,True,False,True,False,169.5,5.138735,False
3,1,-0.998208,66,23,94,28.1,-0.920763,21,0,False,True,False,False,94.0,4.553877,True
4,0,0.504055,40,35,168,43.1,5.484909,33,1,False,False,True,False,168.0,5.129899,False


학습과 에측에 사용할 데이터셋 만들기

In [4]:
df.columns

Index(['Pregnancies', 'Glucose', 'BloodPressure', 'SkinThickness', 'Insulin',
       'BMI', 'DiabetesPedigreeFunction', 'Age', 'Outcome', 'Pregnancies_high',
       'Age_low', 'Age_middle', 'Age_high', 'Insulin_nan', 'Insulin_log',
       'low_glu_insulin'],
      dtype='object')

In [5]:
X = df[[ 'Glucose', 'BloodPressure', 'SkinThickness', 
       'BMI', 'DiabetesPedigreeFunction', 'Age',  'Pregnancies_high', 'Insulin_nan', 
       'low_glu_insulin']]
X.shape

(768, 9)

In [6]:
y = df['Outcome']
y.shape

(768,)

In [7]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(
X, y, test_size=0.2, random_state=42)

In [8]:
X_train.shape, y_train.shape

((614, 9), (614,))

In [9]:
X_test.shape, y_test.shape

((154, 9), (154,))

모데링

In [10]:
#single tree
#from sklearn.tree import DecisionTreeClassifier
#model = DecisionTreeClassifier(random_state=42)
#model

In [11]:
#bagging
#from sklearn.ensemble import RandomForestClassifier

#model = RandomForestClassifier(n_estimators=100 , random_state=42)
#model

In [12]:
#boosting
#from sklearn.ensemble import GradientBoostingClassifier

#model = GradientBoostingClassifier(random_state=42)
#model

Random Search

In [13]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

estimators = [DecisionTreeClassifier(random_state=42),
RandomForestClassifier(random_state=42),
GradientBoostingClassifier(random_state=42)
]
estimators

[DecisionTreeClassifier(random_state=42),
 RandomForestClassifier(random_state=42),
 GradientBoostingClassifier(random_state=42)]

In [16]:
result = []
for estimator in estimators:
    result.append(estimator.__class__.__name__)
result

['DecisionTreeClassifier',
 'RandomForestClassifier',
 'GradientBoostingClassifier']

In [17]:
from sklearn.model_selection import RandomizedSearchCV

max_depth = np.random.randint(2, 20, 10)
max_features = np.random.uniform(0.3, 1.0, 10)

param_distributions = {"max_depth": max_depth,
"max_features": max_features}

results = []
for estimator in estimators:
    result = []
    if estimator.__class__.__name__ != 'DecisionTreeClassifier':
        param_distributions["n_estimators"] = np.random.randint(100, 200, 10)
        
    clf = RandomizedSearchCV(estimator, 
                       param_distributions, 
                       n_iter=100,
                       scoring="accuracy",
                       n_jobs=-1,
                       cv=5, 
                       verbose=2
                      )

    clf.fit(X_train, y_train)
    result.append(estimator.__class__.__name__)
    result.append(clf.best_params_)
    result.append(clf.best_score_)
    result.append(clf.score(X_test, y_test))
    result.append(clf.cv_results_)
    results.append(result)

Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    6.4s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:    9.0s finished
[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    5.8s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   25.8s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:   59.1s
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  1.4min finished


Fitting 5 folds for each of 100 candidates, totalling 500 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:   59.7s
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  2.2min
[Parallel(n_jobs=-1)]: Done 500 out of 500 | elapsed:  3.2min finished


In [18]:
df = pd.DataFrame(results, 
             columns=["estimator", "best_params", "train_score", "test_score", "cv_result"])
df

,estimator,best_params,train_score,test_score,cv_result
0,DecisionTreeClassifier,"{'max_features': 0.3553526763105135, 'max_dept...",0.859963,0.857143,"{'mean_fit_time': [0.012196922302246093, 0.012..."
1,RandomForestClassifier,"{'n_estimators': 199, 'max_features': 0.629697...",0.905611,0.863636,"{'mean_fit_time': [0.46675987243652345, 0.7750..."
2,GradientBoostingClassifier,"{'n_estimators': 132, 'max_features': 0.544070...",0.905598,0.857143,"{'mean_fit_time': [1.4165439128875732, 2.19760..."


In [20]:
pd.DataFrame(df.loc[1, "cv_result"]).sort_values(by="rank_test_score").head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_n_estimators,param_max_features,param_max_depth,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
91,1.150542,0.173028,0.087675,0.024231,199,0.629697,14,"{'n_estimators': 199, 'max_features': 0.629697...",0.878049,0.934959,0.861789,0.902439,0.950820,0.905611,0.033452,1
98,0.953901,0.015442,0.061980,0.006954,199,0.734603,13,"{'n_estimators': 199, 'max_features': 0.734602...",0.869919,0.934959,0.861789,0.910569,0.950820,0.905611,0.034997,1
1,0.775077,0.010990,0.046385,0.003007,175,0.731587,11,"{'n_estimators': 175, 'max_features': 0.731586...",0.861789,0.934959,0.869919,0.910569,0.942623,0.903972,0.032975,3
80,0.575815,0.018723,0.033790,0.001327,130,0.731587,11,"{'n_estimators': 130, 'max_features': 0.731586...",0.861789,0.934959,0.869919,0.910569,0.942623,0.903972,0.032975,3
79,0.574016,0.016088,0.035589,0.003321,130,0.734603,11,"{'n_estimators': 130, 'max_features': 0.734602...",0.861789,0.934959,0.869919,0.910569,0.942623,0.903972,0.032975,3
